In [15]:
import os
import json
import pandas as pd
import traceback

In [20]:
from langchain.chat_models import ChatOpenAI

In [21]:
from dotenv import load_dotenv
load_dotenv()

True

In [22]:
KEY=os.getenv('OPENAI_API_KEY')

In [23]:
print(KEY)

sk-W172123456789012345678901234567890123456789012345678901234567890


In [24]:
llm=ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)

In [25]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000206C1C5F4F0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000206C1CC57B0>, temperature=0.5, model_kwargs={}, openai_api_key='sk-W172123456789012345678901234567890123456789012345678901234567890', openai_proxy='')

In [40]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import  get_openai_callback
import PyPDF2

In [41]:
RESPONSE_JSON = {
  "1": {
    "mcq": "multiple choice question",
    "options": {
      "a": "choice here",
      "b": "choice here",
      "c": "choice here",
      "d": "choice here",
    },
    "correct": "correct answer",
  },
  "2": {
    "mcq": "multiple choice question",
    "options": {
      "a": "choice here",
      "b": "choice here",
      "c": "choice here",
      "d": "choice here",
    },
    "correct": "correct answer",
  },
  "3": {
    "mcq": "multiple choice question",
    "options": {
      "a": "choice here",
      "b": "choice here",
      "c": "choice here",
      "d": "choice here",
    },
    "correct": "correct answer",
  },
}

In [42]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [43]:
quiz_generation_prompt = PromptTemplate(
  input_variables=["text", "number", "subject", "tone", "response_json"],
  template=TEMPLATE
)

In [44]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [45]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [46]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [47]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [48]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["subject", "text", "number", "tone", "response_json"], output_variables=["quiz", "review"], verbose=True)

In [49]:
file_path=r"C:/Users/yfara/OneDrive/Desktop/AI_Development/MCQGen/data.txt"

In [37]:
with open(file_path, 'r') as file:
  TEXT = file.read()

In [50]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [52]:
NUMBER=5
SUBJECT="Biology"
TONE="simple"

In [54]:
with get_openai_callback() as cb:
  response=generate_evaluate_chain(
    {
      "text": TEXT,
      "number": NUMBER,
      "subject":SUBJECT,
      "tone": TONE,
      "response_json": json.dumps(RESPONSE_JSON)
    }
  )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Advances in the field of deep learning have allowed neural networks to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]

From a theoretical viewpoint, probabl

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-W1721*******************************************************7890. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [55]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'01_08_2025_21_51_30'